# Import & Read

In [1]:
import os

In [2]:
os.chdir("E:/dibimbing/Portfolio/Bank-Loan-Default-Prediction")

In [3]:
%pwd

'E:\\dibimbing\\Portfolio\\Bank-Loan-Default-Prediction'

In [4]:
import pandas as pd
import numpy as np

In [5]:
loans = pd.read_csv("artifacts/data_ingestion/SBAnational.csv")
loans.head()

C:\Users\juliu\AppData\Local\Temp\ipykernel_30276\326183299.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  loans = pd.read_csv("artifacts/data_ingestion/SBAnational.csv")


,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,...,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,...,N,Y,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28-Feb-97,1997,...,N,N,NaN,31-Dec-97,"$287,000.00",$0.00,P I F,$0.00,"$287,000.00","$215,250.00"
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28-Feb-97,1997,...,N,Y,NaN,30-Jun-97,"$35,000.00",$0.00,P I F,$0.00,"$35,000.00","$28,000.00"
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28-Feb-97,1997,...,N,N,NaN,14-May-97,"$229,000.00",$0.00,P I F,$0.00,"$229,000.00","$229,000.00"


In [6]:
loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899164 entries, 0 to 899163
Data columns (total 27 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   LoanNr_ChkDgt      899164 non-null  int64  
 1   Name               899150 non-null  object 
 2   City               899134 non-null  object 
 3   State              899150 non-null  object 
 4   Zip                899164 non-null  int64  
 5   Bank               897605 non-null  object 
 6   BankState          897598 non-null  object 
 7   NAICS              899164 non-null  int64  
 8   ApprovalDate       899164 non-null  object 
 9   ApprovalFY         899164 non-null  object 
 10  Term               899164 non-null  int64  
 11  NoEmp              899164 non-null  int64  
 12  NewExist           899028 non-null  float64
 13  CreateJob          899164 non-null  int64  
 14  RetainedJob        899164 non-null  int64  
 15  FranchiseCode      899164 non-null  int64  
 16  Ur

# Cleaning

In [7]:
# Drop null values from specified columns
loans.dropna(subset=['Name', 'City', 'State', 'BankState', 'NewExist','RevLineCr', 'LowDoc', 'DisbursementDate', 'MIS_Status'], inplace=True)

In [8]:
loans.info()

<class 'pandas.core.frame.DataFrame'>
Index: 886240 entries, 0 to 899163
Data columns (total 27 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   LoanNr_ChkDgt      886240 non-null  int64  
 1   Name               886240 non-null  object 
 2   City               886240 non-null  object 
 3   State              886240 non-null  object 
 4   Zip                886240 non-null  int64  
 5   Bank               886240 non-null  object 
 6   BankState          886240 non-null  object 
 7   NAICS              886240 non-null  int64  
 8   ApprovalDate       886240 non-null  object 
 9   ApprovalFY         886240 non-null  object 
 10  Term               886240 non-null  int64  
 11  NoEmp              886240 non-null  int64  
 12  NewExist           886240 non-null  float64
 13  CreateJob          886240 non-null  int64  
 14  RetainedJob        886240 non-null  int64  
 15  FranchiseCode      886240 non-null  int64  
 16  UrbanRu

In [9]:
# Define a function to clean the string from integer data
def clean_data_str(value):
    if isinstance(value, str):
        return value.replace('A', '')
    return value

loans['ApprovalFY'] = loans['ApprovalFY'].apply(clean_data_str).astype('int64')

In [10]:
# Remove '$', commas, and extra spaces from records in columns with dollar values that should be floats

loans[['DisbursementGross', 'BalanceGross', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv']] = \
loans[['DisbursementGross', 'BalanceGross', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv']].map(lambda x: x.strip().replace('$', '').replace(',', ''))

In [11]:
# Reconfigure remaining data into appropriate data types.
# Here i change data types of Zip, NewExist, UrbanRural and all currency types to the appropriate data types.
loans = loans.astype({
    'Zip': 'str',
    'NewExist': 'int64',
    'UrbanRural': 'str',
    'DisbursementGross' : 'float64',
    'BalanceGross' : 'float64',
    'ChgOffPrinGr' : 'float64',
    'GrAppv' : 'float64',
    'SBA_Appv' : 'float64'
    })

In [12]:
# Mapping dictionary for NAICS codes
naics_mapping = {
    '11': 'Agri/For/Fish/Hunt', # 'Agriculture, forestry, fishing and hunting',
    '21': 'Min/Quar/Oil/Gas_ext', # 'Mining, quarrying, and oil and gas extraction',
    '22': 'Utilities', # 'Utilities',
    '23': 'Construction', # 'Construction',
    '31': 'Manufacturing', # 'Manufacturing',
    '32': 'Manufacturing', # 'Manufacturing',
    '33': 'Manufacturing', # 'Manufacturing',
    '42': 'Wholesale_trade', # 'Wholesale trade',
    '44': 'Retail_trade', # 'Retail trade',
    '45': 'Retail_trade', # 'Retail trade',
    '48': 'Transp/WareH', # 'Transportation and warehousing',
    '49': 'Transp/WareH', # 'Transportation and warehousing',
    '51': 'Info', # 'Information',
    '52': 'Finance/Insurance', # 'Finance and insurance',
    '53': 'Real_est/Rental/Lease', # 'Real estate and rental and leasing',
    '54': 'Prof/Science/Tech', # 'Professional, scientific, and technical services',
    '55': 'Mgmt_comp_entp', # 'Management of companies and enterprises',
    '56': 'Admin/Supp/WasteM/Remed', # 'Administrative and support and waste management and remediation services',
    '61': 'Edu', # 'Educational services',
    '62': 'Health/Social', # 'Health care and social assistance',
    '71': 'Art/Entr/Rec', # 'Arts, entertainment, and recreation',
    '72': 'Accom/Food', # 'Accommodation and food services',
    '81': 'Other', # 'Other services (except public administration)',
    '92': 'Pub_Admin', # 'Public administration'
}

# Function to map NAICS codes to sectors (created two, one for Industry Sector text and one for Industry Code)
loans['Industry'] = loans['NAICS'].astype('str').apply(lambda x: x[:2])
loans['IndustryCode'] = loans['NAICS'].astype('str').apply(lambda x: x[:2])

# map Industry code based on mapping
loans['Industry'] = loans['Industry'].map(naics_mapping)

# Remove records where Industry is NaN (NAICS code was a 0)
loans.dropna(subset=['Industry'], inplace=True)


In [13]:
loans[['NAICS','Industry']].head()

,NAICS,Industry
0,451120,Retail_trade
1,722410,Accom/Food
2,621210,Health/Social
5,332721,Manufacturing
7,811118,Other


In [14]:
loans = loans[(loans['NewExist'] == 1) | (loans['NewExist'] == 2)]

# I then create a new column where NewBusiness = 0 is an existing business, and NewBusiness = 1 is a new business
loans.loc[(loans['NewExist'] == 1), 'NewBusiness'] = 0
loans.loc[(loans['NewExist'] == 2), 'NewBusiness'] = 1

In [15]:
print(loans['RevLineCr'].unique())
print(loans['LowDoc'].unique())

['N' 'Y' '0' 'T' '`' ',' '1' 'C' '2' 'R' '7' 'A' '-' 'Q']
['Y' 'N' 'C' '1' 'S' 'R' 'A' '0']


In [16]:
# Temove records where RevLineCr != Y or N and LowDoc != Y or N
loans = loans[(loans['RevLineCr'] == 'Y') | (loans['RevLineCr'] == 'N')]
loans = loans[(loans['LowDoc'] == 'Y') | (loans['LowDoc'] == 'N')]

# Transform data to int
pd.set_option('future.no_silent_downcasting', True) #future-proof
loans['RevLineCr'] = loans['RevLineCr'].replace(['N', 'Y'], [0, 1])
loans['LowDoc'] = loans['LowDoc'].replace(['N' ,'Y'],[0,1])

print(loans['RevLineCr'].unique())
print(loans['LowDoc'].unique())

[0 1]
[1 0]


In [17]:
# Created a flag for FranchiseCode = IsFranchise
loans.loc[(loans['FranchiseCode'] <= 1), 'IsFranchise'] = 0
loans.loc[(loans['FranchiseCode'] > 1), 'IsFranchise'] = 1

In [18]:
# Convert ApprovalDate and DisbursementDate columns to datetime values
# ChgOffDate not changed to datetime since it is not relevant to the analysis since it only states when the approval is charged off and will be removed later

loans[['ApprovalDate', 'DisbursementDate']] = loans[['ApprovalDate', 'DisbursementDate']].apply(pd.to_datetime)

C:\Users\juliu\AppData\Local\Temp\ipykernel_30276\3596729647.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  loans[['ApprovalDate', 'DisbursementDate']] = loans[['ApprovalDate', 'DisbursementDate']].apply(pd.to_datetime)
C:\Users\juliu\AppData\Local\Temp\ipykernel_30276\3596729647.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  loans[['ApprovalDate', 'DisbursementDate']] = loans[['ApprovalDate', 'DisbursementDate']].apply(pd.to_datetime)


In [19]:
print(loans[['ApprovalDate', 'DisbursementDate']].head())

  ApprovalDate DisbursementDate
0   1997-02-28       1999-02-28
1   1997-02-28       1997-05-31
2   1997-02-28       1997-12-31
5   1997-02-28       1997-06-30
7   1997-02-28       1998-06-30


In [20]:
# Create DaysToDisbursement column calculating days between DisbursementDate and ApprovalDate
loans['DaysToDisbursement'] = (loans['DisbursementDate'] - loans['ApprovalDate']).dt.days

# Convert DaysToDisbursement to int64 dtype
loans['DaysToDisbursement'] = loans['DaysToDisbursement'].astype('int64')

# Create DisbursementMonth & DisbursementYear field for time analysis later (Great Recession categorizing)
loans['DisbursementMonth'] = loans['DisbursementDate'].map(lambda x: x.month)
loans['DisbursementYear'] = loans['DisbursementDate'].map(lambda x: x.year)

In [21]:
# Create StateSame flag field which identifies where the business State is the same as the BankState
loans['StateSame'] = np.where(loans['State'] == loans['BankState'], 1, 0)

In [22]:
# Percent = SBA_Appv / GrAppv

loans['SBA_AppvPercent'] = loans['SBA_Appv'] / loans['GrAppv']

In [23]:
# Create AppvDisbursed flag field signifying if the loan amount disbursed was equal to the full amount approved
loans['AppvDisbursed'] = np.where(loans['DisbursementGross'] == loans['GrAppv'], 1, 0)

In [24]:
# Field for loans backed by Real Estate (loans with a term of at least 20 years)
loans['RealEstate'] = np.where(loans['Term'] >= 240, 1, 0)


# Column for loans through economic recession
loans['GreatRecession'] = np.where(
    (
        (loans['DisbursementYear'] == 2007) & (loans['DisbursementMonth'] >= 12) |
        (loans['DisbursementYear'] == 2008) |
        (loans['DisbursementYear'] == 2009) & (loans['DisbursementMonth'] <= 6)
    ),
    1,0)

In [25]:
loans = loans.astype({
    'IsFranchise': 'int64',
    'NewBusiness': 'int64',
    'RevLineCr': 'int64',
    'LowDoc': 'int64'
    })

In [26]:
# Create a new `Default` column to represent MIS_Status where P I F = 0 and CHGOFF = 1
loans['Default'] = np.where(loans['MIS_Status'] == 'P I F', 0, 1)

In [27]:
loans.info()

<class 'pandas.core.frame.DataFrame'>
Index: 456940 entries, 0 to 899161
Data columns (total 40 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   LoanNr_ChkDgt       456940 non-null  int64         
 1   Name                456940 non-null  object        
 2   City                456940 non-null  object        
 3   State               456940 non-null  object        
 4   Zip                 456940 non-null  object        
 5   Bank                456940 non-null  object        
 6   BankState           456940 non-null  object        
 7   NAICS               456940 non-null  int64         
 8   ApprovalDate        456940 non-null  datetime64[ns]
 9   ApprovalFY          456940 non-null  int64         
 10  Term                456940 non-null  int64         
 11  NoEmp               456940 non-null  int64         
 12  NewExist            456940 non-null  int64         
 13  CreateJob           456940 non-nul

# Preprocessing

In [28]:
# Copy data for model

loans_model_df = loans.copy()

In [29]:
state_mapping = {state: idx for idx, state in enumerate([
    'IN', 'OK', 'FL', 'CT', 'NJ', 'NC', 'IL', 'RI', 'TX', 'VA',
    'TN', 'AR', 'MN', 'MO', 'MA', 'CA', 'SC', 'LA', 'IA', 'OH',
    'KY', 'MS', 'NY', 'MD', 'PA', 'OR', 'ME', 'KS', 'MI', 'AK',
    'WA', 'CO', 'MT', 'WY', 'UT', 'NH', 'WV', 'ID', 'AZ', 'NV',
    'WI', 'NM', 'GA', 'ND', 'VT', 'AL', 'NE', 'SD', 'HI', 'DE', 'DC'
])}

bank_state_mapping = {state: idx for idx, state in enumerate([
    'OH', 'IN', 'OK', 'FL', 'DE', 'SD', 'AL', 'CT', 'GA', 'OR', 'MN', 'RI', 'NC', 'TX',
    'MD', 'NY', 'TN', 'SC', 'MS', 'MA', 'LA', 'IA', 'VA', 'CA', 'IL', 'KY', 'PA', 'MO',
    'WA', 'MI', 'UT', 'KS', 'WV', 'WI', 'AZ', 'NJ', 'CO', 'ME', 'NH', 'AR', 'ND', 'MT',
    'ID', 'WY', 'NM', 'DC', 'NV', 'NE', 'PR', 'HI', 'VT', 'AK', 'GU', 'AN', 'EN', 'VI'
  ])}

In [30]:
# Drop respected columns

loans_model_df.drop(columns=['LoanNr_ChkDgt', 'ChgOffDate', 'Name', 'City', 'Zip', 'Bank', 'NAICS', 'MIS_Status', 'NewExist', 'FranchiseCode',
                      'ApprovalDate', 'DisbursementDate'], inplace=True)

In [31]:
# Encoding like base model encoding

loans_model_df['State'] = loans_model_df['State'].map(state_mapping)
loans_model_df['BankState'] = loans_model_df['BankState'].map(bank_state_mapping)

In [32]:
loans_model_df = loans_model_df.astype({
    'IndustryCode' : 'int64',
    'UrbanRural': 'int64'
    })

# MLOPS Prep

In [33]:
loans_model_df.drop(columns=['Industry'], inplace=True)

In [34]:
loans_model_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 456940 entries, 0 to 899161
Data columns (total 27 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   State               456940 non-null  int64  
 1   BankState           456940 non-null  int64  
 2   ApprovalFY          456940 non-null  int64  
 3   Term                456940 non-null  int64  
 4   NoEmp               456940 non-null  int64  
 5   CreateJob           456940 non-null  int64  
 6   RetainedJob         456940 non-null  int64  
 7   UrbanRural          456940 non-null  int64  
 8   RevLineCr           456940 non-null  int64  
 9   LowDoc              456940 non-null  int64  
 10  DisbursementGross   456940 non-null  float64
 11  BalanceGross        456940 non-null  float64
 12  ChgOffPrinGr        456940 non-null  float64
 13  GrAppv              456940 non-null  float64
 14  SBA_Appv            456940 non-null  float64
 15  IndustryCode        456940 non-null  in

In [35]:
loans_model_df.columns

Index(['State', 'BankState', 'ApprovalFY', 'Term', 'NoEmp', 'CreateJob',
       'RetainedJob', 'UrbanRural', 'RevLineCr', 'LowDoc', 'DisbursementGross',
       'BalanceGross', 'ChgOffPrinGr', 'GrAppv', 'SBA_Appv', 'IndustryCode',
       'NewBusiness', 'IsFranchise', 'DaysToDisbursement', 'DisbursementMonth',
       'DisbursementYear', 'StateSame', 'SBA_AppvPercent', 'AppvDisbursed',
       'RealEstate', 'GreatRecession', 'Default'],
      dtype='object')

In [36]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [37]:
from src.loanDefault.constants import *
from src.loanDefault.utils.common import read_yaml, create_directories

In [38]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [39]:
import os
from src.loanDefault import logger

In [40]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e



In [41]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2024-10-01 20:16:27,779: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-01 20:16:27,783: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-01 20:16:27,787: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-10-01 20:16:27,789: INFO: common: created directory at: artifacts]
[2024-10-01 20:16:27,790: INFO: common: created directory at: artifacts/data_validation]


C:\Users\juliu\AppData\Local\Temp\ipykernel_30276\63668955.py:10: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(self.config.unzip_data_dir)
